In [ ]:
import numpy as np
from pyproj import Transformer

# Load XYZ file (columns: X Y Z)
xyz = np.loadtxt("file.xyz")
x, y, z = xyz[:,0], xyz[:,1], xyz[:,2]

# Define CRS
src_crs = "EPSG:31255"   # original CRS
dst_crs = "EPSG:4326"    # target CRS (WGS84 lat/lon)

# Create transformer
transformer = Transformer.from_crs(src_crs, dst_crs, always_xy=True)

# Reproject coordinates
lon, lat = transformer.transform(x, y)

# Combine reprojected coordinates with original Z
xyz_reprojected = np.column_stack((lon, lat, z))

# Save to new file
np.savetxt("file_wgs84.xyz", xyz_reprojected, fmt="%.6f")


In [ ]:
import numpy as np
import rasterio
from rasterio.transform import from_origin
from rasterio.warp import calculate_default_transform, reproject, Resampling

# Load XYZ
xyz = np.loadtxt("file.xyz")
x, y, z = xyz[:,0], xyz[:,1], xyz[:,2]

# Assuming gridded data
x_unique = np.sort(np.unique(x))
y_unique = np.sort(np.unique(y))[::-1]  # flip Y
dx = np.unique(np.diff(x_unique))[0]
dy = np.unique(np.diff(y_unique))[0]
z_grid = z.reshape(len(y_unique), len(x_unique))

# GeoTIFF transform
transform = from_origin(x_unique[0], y_unique[0], dx, dy)

# Write original GeoTIFF
with rasterio.open(
    "output_31255.tif",
    "w",
    driver="GTiff",
    height=z_grid.shape[0],
    width=z_grid.shape[1],
    count=1,
    dtype=z_grid.dtype,
    crs="EPSG:31255",
    transform=transform
) as dst:
    dst.write(z_grid, 1)

# Reproject to WGS84
with rasterio.open("output_31255.tif") as src:
    transform, width, height = calculate_default_transform(
        src.crs, 'EPSG:4326', src.width, src.height, *src.bounds)
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': 'EPSG:4326',
        'transform': transform,
        'width': width,
        'height': height
    })
    with rasterio.open('output_wgs84.tif', 'w', **kwargs) as dst:
        reproject(
            source=rasterio.band(src, 1),
            destination=rasterio.band(dst, 1),
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=transform,
            dst_crs='EPSG:4326',
            resampling=Resampling.bilinear
        )
